- Importing Libraries

In [1]:
import torch
from transformers import BertModel
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
from transformers import BertTokenizer
import string

### Load Dataset for the Particular Language
- Here insert the language initial you want to load the train and test CSVs for
- Language Codes:
    1. Hindi: `hi`
    2. Bengali: `bn`
    3. Marathi: `mr`
    4. Tamil: `ta`
    5. Telugu: `te`

In [2]:
# Define file paths for two languages
languages = ['hi', 'mr']
train_files = [f"/kaggle/input/dataset/Dataset/{lang}_train.csv" for lang in languages]
test_files = [f"/kaggle/input/dataset/Dataset/{lang}_test.csv" for lang in languages]

# Concatenate train datasets for two languages
train_dfs = []
for train_file in train_files:
    train_dfs.append(pd.read_csv(train_file))
train_df = pd.concat(train_dfs, ignore_index=True)

# Concatenate test datasets for two languages
test_dfs = []
for test_file in test_files:
    test_dfs.append(pd.read_csv(test_file))
test_df = pd.concat(test_dfs, ignore_index=True)

print("Length of Concatenated Train Dataset:", len(train_df))
print("Length of Concatenated Test Dataset:", len(test_df))

Length of Concatenated Train Dataset: 2000
Length of Concatenated Test Dataset: 312


## Custom Dataset Class
- This class can be used for all Sentiment Analysis fine-tuning and zero shot tasks
- For multiple languages, combine data into one dataframe as the dataset class takes DATAFRAME as input

In [3]:
class MultilingualSentimentAnalysis_Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        label_map = {"Negative": 0, "Positive": 1}

        label_value = self.dataframe.iloc[idx]["LABEL"]
        if pd.isna(label_value):
            label = 0
        else:
            label = label_map[label_value]


        input_text = self.dataframe.iloc[idx]["INDIC REVIEW"]

        # Tokenize
        encoding = self.tokenizer.encode_plus(
            input_text, None,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }

In [4]:
# Get the model name
model_name = "bert-base-multilingual-cased"

# Initialise the tokeniser
tokenizer = BertTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [5]:
# Initialise dataset instances
train_dataset = MultilingualSentimentAnalysis_Dataset(train_df, tokenizer)
dev_dataset = MultilingualSentimentAnalysis_Dataset(test_df, tokenizer)

# Initialise the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=False)

In [6]:
import torch
import torch.nn as nn

class MBertForSentimentAnalysis(nn.Module):
    def __init__(self, freeze_bert=False):
        super(MBertForSentimentAnalysis, self).__init__()

        # Load mBERT model and tokenizer
        self.model_name = "bert-base-multilingual-cased"
        # tokenizer = BertTokenizer.from_pretrained(model_name)
        self.mbert = BertModel.from_pretrained(self.model_name)

        # Add a batch normalization layer
        self.batch_norm = nn.BatchNorm1d(self.mbert.config.hidden_size)

        # Add a linear layer for classification
        self.classification = nn.Linear(self.mbert.config.hidden_size, 2)

        # Option to freeze MBERT layers to prevent them from being updated during training
        if freeze_bert:
            for param in self.mbert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        # Get the output from BERT model
        _, pooled_outputs = self.mbert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)

        # Pass output through batch normalization layer
        pooled_outputs = self.batch_norm(pooled_outputs)

        # Pass output through linear layer
        out = self.classification(pooled_outputs)
        return out

In [7]:
model = MBertForSentimentAnalysis()
if torch.cuda.is_available():
    device = torch.device("cuda")
    model.to(device)  # Move model to CUDA device if available
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU instead.")

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Using CUDA


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
ce_loss = torch.nn.CrossEntropyLoss()

num_epochs = 4

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        # Forward pass
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        loss = ce_loss(outputs, inputs['labels'].long())

        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():  # No need to compute gradients during validation
        for batch in dev_dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
            loss = ce_loss(outputs, inputs['labels'].long())
            val_loss += loss.item()

    # Calculate average losses
    avg_train_loss = train_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(dev_dataloader)

    # Append the losses for plotting
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

Epoch 1/4, Train Loss: 0.4913, Validation Loss: 0.3813
Epoch 2/4, Train Loss: 0.3013, Validation Loss: 0.4225
Epoch 3/4, Train Loss: 0.1990, Validation Loss: 0.4005
Epoch 4/4, Train Loss: 0.1136, Validation Loss: 0.3768


### Evaluation of Fine-Tuned Model
- Evaluate fine-tuned model on test dataset of each language

In [9]:
# Load datasets for each language
languages = ["hi", "mr", "bn", "ta", "te"]

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

with torch.no_grad():
    for lang in languages:
        test_file = f"/kaggle/input/dataset/Dataset/{lang}_test.csv"
        test_df = pd.read_csv(test_file)
        test_dataloader = DataLoader(MultilingualSentimentAnalysis_Dataset(test_df, tokenizer), batch_size=16, shuffle=True)

        # Make list for predicted labels and ground truth labels
        predicted_labels = []
        labels = []

        # Perform inference
        for batch in test_dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
            predicted_labels.extend(torch.argmax(outputs, dim=1).tolist())
            labels.extend(inputs['labels'].long().tolist())


        # Print results for a particular language
        print(f"RESULTS FOR {lang}")
        print()
        # Calculate accuracy
        accuracy = accuracy_score(labels, predicted_labels)
        print(f'Accuracy: {accuracy:.4f}')

        # Calculate F1-score
        weighted_f1_score = f1_score(labels, predicted_labels, average='weighted')
        macro_f1_score = f1_score(labels, predicted_labels, average='macro')
        print(f'Weighted F1-score: {weighted_f1_score:.4f}')
        print(f'Macro F1-score: {macro_f1_score:.4f}')
        print()

        # Print classification report
        print("Classification Report")
        print(classification_report(labels, predicted_labels))
        print()
        # Print confusion matrix
        print("Confusion Matrix")
        print(confusion_matrix(labels, predicted_labels))
        print()
        print("----------*******************---------------")


RESULTS FOR hi

Accuracy: 0.8782
Weighted F1-score: 0.8779
Macro F1-score: 0.8781

Classification Report
              precision    recall  f1-score   support

           0       0.94      0.81      0.87        81
           1       0.83      0.95      0.88        75

    accuracy                           0.88       156
   macro avg       0.88      0.88      0.88       156
weighted avg       0.89      0.88      0.88       156


Confusion Matrix
[[66 15]
 [ 4 71]]

----------*******************---------------
RESULTS FOR mr

Accuracy: 0.8590
Weighted F1-score: 0.8587
Macro F1-score: 0.8584

Classification Report
              precision    recall  f1-score   support

           0       0.85      0.89      0.87        81
           1       0.87      0.83      0.85        75

    accuracy                           0.86       156
   macro avg       0.86      0.86      0.86       156
weighted avg       0.86      0.86      0.86       156


Confusion Matrix
[[72  9]
 [13 62]]

----------*****

In [10]:
# Save Model and Tokenizer
model_save_path = "MBERT_SA_Hi-Mr_FineTune.pth"
torch.save(model.state_dict(), model_save_path)